# Enoncé

Vous avez participé à une compétition sur Kaggle sur le jeu de données de Titanic (celle-ci existe, les curieux peuvent la retrouver sur Kaggle !). Vous avez pour cela à votre disposition [une liste de 891 passagers](https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv), contenant les caractéristiques suivantes :

- PassengerID : Identifiant du passager
- Survived : Indicateur de survi d'un passager (1 si le passager a survecu, 0 s’il est décédé)
- Pclass: Classe du passager (1 = 1ère classe, 2 = 2ème classe, 3 = 3ème classe)
- Name : Nom et titre du passager
- Sex : Sexe du passager
- Age : Age du passager (Décimal si inférieur à 1, estimé si de la forme xx.5)
- SibSp : Nombre d’époux, de frères et de soeurs présents à bord
- Parch : Nombre de parents ou d’enfants présents à bord 
- Ticket : Numéro du ticket 
- Fare : Tarif des tickets (Le prix est indiqué en £ et pour un seul achat (peut correspondre à plusieurs tickets)
- Cabin : Numéro de Cabine
- Embarked : Port d’embarcation (C = Cherbourg, Q = Queenstown, S = Southampton)

In [8]:
import pandas as pd
import numpy as np
from os import getcwd
from sklearn.neighbors import KNeighborsClassifier

## 1. Charger vos données dans un DataFrame Pandas

In [9]:
# ---------------------------------------------------------------------------------------------
#                               MAIN
# ---------------------------------------------------------------------------------------------
verbose = False
verboseMain = False

print("Chargement des données...")
# Récupère le répertoire du programme
file_path = getcwd() + "\\"

Chargement des données...


In [10]:
file_name = 'titanic_dataset.csv'

df_origin = pd.read_csv(file_path+file_name, sep=',', index_col="PassengerId")

print("Chargement des données", df_origin.shape, ".... ...................................... END")

Chargement des données (891, 11) .... ...................................... END


In [11]:
df_origin.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
df = df_origin.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [18]:
df.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [19]:
# display 
df.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

 ## 2. Nettoyer votre Dataset avec drop(), dropna() 

**Supprimer les variables 'PassengerId','Name','SibSp','Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'**

Vérifiez la taille de votre data, supprimer les valeurs manquantes

In [21]:
print("before",df.shape, end="")
df = df.drop(['Name','SibSp','Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],axis=1)
print(" drop col :",df.shape, end="")
df.dropna(subset=['Age', 'Sex'])
print(" drop na :",df.shape)

before (891, 11) drop col : (891, 4) drop na : (891, 4)


In [27]:
df = df.dropna(subset=['Age', 'Sex'])
print(" after",df.shape)

 after (714, 4)


# Exercice

La compétition a été l’occasion de revenir sur ce jeu de données très célèbre, et plusieurs tâches étaient attendues, :
- identifier les facteurs favorisants la survie d'un passager par rapport à un autre, en dressant une typologie des survivants
- créer un algorithme qui pourrait prédire la survie d'un individu à partir de ces caractéristiques.

Vous avez été ajouté à une équipe et le travail et lancé depuis quelques semaines : à cette étape,  la mission est en réalité finie (c.f. le présent notebook). Vos co-équipiers ont travaillé dur : il faut dans un premier temps vous approprier leur travail.

In [28]:
titanic = df[["Survived", "Pclass", "Sex", "Age" ]]
titanic.head()

,Survived,Pclass,Sex,Age
PassengerId,,,,
1,0,3,male,22.0
2,1,1,female,38.0
3,1,3,female,26.0
4,1,1,female,35.0
5,0,3,male,35.0


In [29]:
titanic["Sex"].replace(["male", "female"], [0,1], inplace=True)
titanic.head()

,Survived,Pclass,Sex,Age
PassengerId,,,,
1,0,3,0,22.0
2,1,1,1,38.0
3,1,3,1,26.0
4,1,1,1,35.0
5,0,3,0,35.0


In [30]:
titanic.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
dtype: int64

In [41]:
model = KNeighborsClassifier(n_neighbors=1)

In [42]:
y = titanic["Survived"]
x = titanic.drop("Survived", axis=1)
print(y.head())
x.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64


,Pclass,Sex,Age
PassengerId,,,
1,3,0,22.0
2,1,1,38.0
3,3,1,26.0
4,1,1,35.0
5,3,0,35.0


In [43]:
model.fit(x, y)
model.score(x,y)

0.8739495798319328

In [44]:

def survie(model, pclass=3, sex=1, age=40):
    x = np.array([pclass, sex, age]).reshape(1, 3)
    print(model.predict(x))
    print(model.predict_proba(x))

In [45]:
survie(model, 3, 1, 40)

[0]
[[1. 0.]]


C:\Program Files\Python39\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Program Files\Python39\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [39]:
# tester entre 0 et 10

def found_better_neigbors(x, y):
    better_score = 0
    better_n=0
    for i in range (1, 10):
        model = KNeighborsClassifier(n_neighbors=i) 
        model.fit(x, y)
        res = model.score(x,y)
        if res > better_score:
            better_score = res
            better_n = i
    print(f"Avec un score de {better_score}, le meilleur paramètre est {better_n}")
    return better_n

In [40]:
found_better_neigbors(x, y)

Avec un score de 0.8739495798319328, le meilleur paramètre est 1


1